In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [6]:
loss_fn = nn.CrossEntropyLoss()

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')
            
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f'Epoch {t+1}\n---------------------------------')
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print('Done!')

Epoch 1
---------------------------------
loss: 2.305268 [    0/60000]
loss: 2.297215 [ 6400/60000]
loss: 2.270583 [12800/60000]
loss: 2.267398 [19200/60000]
loss: 2.252149 [25600/60000]
loss: 2.220494 [32000/60000]
loss: 2.238076 [38400/60000]
loss: 2.192407 [44800/60000]
loss: 2.189342 [51200/60000]
loss: 2.163422 [57600/60000]
Test Error: 
 Accuracy: 42.6%, Avg loss: 2.159208 

Epoch 2
---------------------------------
loss: 2.169719 [    0/60000]
loss: 2.164446 [ 6400/60000]
loss: 2.102996 [12800/60000]
loss: 2.121479 [19200/60000]
loss: 2.079519 [25600/60000]
loss: 2.015461 [32000/60000]
loss: 2.052312 [38400/60000]
loss: 1.962698 [44800/60000]
loss: 1.968508 [51200/60000]
loss: 1.903664 [57600/60000]
Test Error: 
 Accuracy: 56.4%, Avg loss: 1.901628 

Epoch 3
---------------------------------
loss: 1.935290 [    0/60000]
loss: 1.912402 [ 6400/60000]
loss: 1.791198 [12800/60000]
loss: 1.832976 [19200/60000]
loss: 1.737048 [25600/60000]
loss: 1.677525 [32000/60000]
loss: 1.708888 [